In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch.nn.functional as F
import torchsummary
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = ImageFolder('/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/', transform=train_transforms)
test_dataset = ImageFolder('/kaggle/input/asl-alphabet-modified-test/asl_alphabet_test/asl_alphabet_test/', transform=test_transforms)
unseen_test_dataset = ImageFolder('/kaggle/input/asl-alphabet-test/', transform=test_transforms)

train_size = int(0.8 * len(train_data))
val_size = int(0.2 * len(train_data))
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
unseen_test_loader = DataLoader(unseen_test_dataset, batch_size=64, shuffle=False)

print("Train Dataset Size: ",len(train_dataset))
print("Validation Dataset Size: ",len(val_dataset))
print("Test Dataset Size: ",len(test_dataset))
print("Unseen Test Dataset Size: ",len(unseen_test_dataset))

In [ ]:
# Load a pre-trained DenseNet model
ResNet_model = models.resnet50(pretrained=True)

# Freeze layers
for param in ResNet_model.parameters():
    param.requires_grad = False

# Replace the classifier
num_features = ResNet_model.fc.in_features
ResNet_model.fc = nn.Linear(num_features, 29)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ResNet_model.fc.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ResNet_model = ResNet_model.to(device)

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    ResNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ResNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    ResNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = ResNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (ResNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (ResNet)')
plt.legend()
plt.show()

In [ ]:
# Specify the file path on your Google Drive
file_path = '/kaggle/working/frozen_resnet_model.pth'

# Save the model
torch.save(ResNet_model.state_dict(), file_path)

In [ ]:
for name, param in ResNet_model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = True
        
    if 'layer2' in name:
        param.requires_grad = False
    
    if 'layer3' in name:
        param.requires_grad = False
    
    if 'layer4' in name:
        param.requires_grad = False
optimizer = optim.Adam(ResNet_model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    ResNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ResNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    ResNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = ResNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (ResNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (ResNet)')
plt.legend()
plt.show()

In [ ]:
for name, param in ResNet_model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = True
        
    if 'layer2' in name:
        param.requires_grad = True
    
    if 'layer3' in name:
        param.requires_grad = False
    
    if 'layer4' in name:
        param.requires_grad = False
optimizer = optim.Adam(ResNet_model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    ResNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ResNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    ResNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = ResNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (ResNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (ResNet)')
plt.legend()
plt.show()

In [ ]:
for name, param in ResNet_model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = True
        
    if 'layer2' in name:
        param.requires_grad = True
    
    if 'layer3' in name:
        param.requires_grad = True
    
    if 'layer4' in name:
        param.requires_grad = False
optimizer = optim.Adam(ResNet_model.parameters(), lr=0.000001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    ResNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ResNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    ResNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = ResNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (ResNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (ResNet)')
plt.legend()
plt.show()

In [ ]:
for name, param in ResNet_model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = True
        
    if 'layer2' in name:
        param.requires_grad = True
    
    if 'layer3' in name:
        param.requires_grad = True
    
    if 'layer4' in name:
        param.requires_grad = True
optimizer = optim.Adam(ResNet_model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    ResNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = ResNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    ResNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = ResNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (ResNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (ResNet)')
plt.legend()
plt.show()

In [ ]:
ResNet_model.eval()
with torch.no_grad():
    test_total = 0
    test_correct = 0
    y_true = []
    y_pred = []
    for images, labels in unseen_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = ResNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
    test_acc = (test_correct / test_total)*100
    print('Test Accuracy: {:.2f}%'.format(test_acc))

In [ ]:
ResNet_model.eval()
with torch.no_grad():
    test_total = 0
    test_correct = 0
    y_true = []
    y_pred = []
    for images, labels in unseen_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = ResNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
    test_acc = (test_correct / test_total)*100
    print('Test Accuracy for unseen data: {:.2f}%'.format(test_acc))

In [ ]:
# Specify the file path on your Google Drive
file_path = '/kaggle/working/tuned_resnet_model.pth'

# Save the model
torch.save(ResNet_model.state_dict(), file_path)

In [ ]:
ResNet_model_copy = copy.deepcopy(ResNet_model)

In [ ]:
ResNet_model_summary = torchsummary.summary(ResNet_model, input_size=(3, 200, 200))
print(ResNet_model_summary)

In [ ]:
# Load a pre-trained DenseNet model
DenseNet_model = models.densenet121(pretrained=True)

# Freeze layers
for param in DenseNet_model.parameters():
    param.requires_grad = False

# Replace the classifier
num_features = DenseNet_model.classifier.in_features
DenseNet_model.classifier = nn.Linear(num_features, 29)

# Define loss function and optimizer
optimizer = optim.Adam(DenseNet_model.classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DenseNet_model = DenseNet_model.to(device)

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    DenseNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = DenseNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    DenseNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = DenseNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (DenseNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (DenseNet)')
plt.legend()
plt.show()

In [ ]:
# Specify the file path on your Google Drive
file_path = '/kaggle/working/frozen_densenet_model.pth'

# Save the model
torch.save(DenseNet_model.state_dict(), file_path)

In [ ]:
for name, param in DenseNet_model.named_parameters():
    if 'transition3' in name:
        param.requires_grad = True
        
    if 'denseblock4' in name:
        param.requires_grad = False
    
    if 'norm5' in name:
        param.requires_grad = False
        
optimizer = optim.Adam(DenseNet_model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    DenseNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = DenseNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    DenseNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = DenseNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (DenseNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (DenseNet)')
plt.legend()
plt.show()

In [ ]:
for name, param in DenseNet_model.named_parameters():
    if 'transition3' in name:
        param.requires_grad = True
        
    if 'denseblock4' in name:
        param.requires_grad = True
    
    if 'norm5' in name:
        param.requires_grad = False
        
optimizer = optim.Adam(DenseNet_model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    DenseNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = DenseNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    DenseNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = DenseNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (DenseNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (DenseNet)')
plt.legend()
plt.show()

In [ ]:
for name, param in DenseNet_model.named_parameters():
    if 'transition3' in name:
        param.requires_grad = True
        
    if 'denseblock4' in name:
        param.requires_grad = True
    
    if 'norm5' in name:
        param.requires_grad = True
        
optimizer = optim.Adam(DenseNet_model.parameters(), lr=0.000001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    running_loss = 0.0
    DenseNet_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = DenseNet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = (train_correct / train_total)*100
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate the model on the validation set
    DenseNet_model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = DenseNet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = (val_correct / val_total)*100
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f'Epoch {epoch+1} Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.2f}% Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.2f}%')

plt.plot(range(1, num_epochs+1), train_losses, label='train')
plt.plot(range(1, num_epochs+1), val_losses, label='val')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Loss vs Epoch (DenseNet)')
plt.legend()
plt.show()

plt.plot(range(1, num_epochs+1), train_accs, label='train')
plt.plot(range(1, num_epochs+1), val_accs, label='val')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title(f'Accuracy vs Epoch (DenseNet)')
plt.legend()
plt.show()

In [ ]:
# Specify the file path on your Google Drive
file_path = '/kaggle/working/tuned_densenet_model.pth'

# Save the model
torch.save(DenseNet_model.state_dict(), file_path)

In [ ]:
DenseNet_model_copy = copy.deepcopy(DenseNet_model)

In [ ]:
DenseNet_model.eval()
with torch.no_grad():
    test_total = 0
    test_correct = 0
    y_true = []
    y_pred = []
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = DenseNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
    test_acc = (test_correct / test_total)*100
    print('Test Accuracy: {:.2f}%'.format(test_acc))

In [ ]:
DenseNet_model.eval()
with torch.no_grad():
    test_total = 0
    test_correct = 0
    y_true = []
    y_pred = []
    for images, labels in unseen_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = DenseNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
    test_acc = (test_correct / test_total)*100
    print('Test Accuracy for unseen data: {:.2f}%'.format(test_acc))

In [ ]:
DenseNet_model_summary = torchsummary.summary(DenseNet_model, input_size=(3, 200, 200))
print(DenseNet_model_summary)